In [ ]:
import osmnx as ox
import networkx as nx
import pickle 
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chisquare
import matplotlib.patches as mpatches
from scipy.sparse import spdiags
from scipy.linalg import eigh
from sklearn.cluster import KMeans
from scipy.stats import zscore
import matplotlib.pyplot as plt

path = 'C:/Users/Besitzer/Desktop/Geospatial Analysis/Data'

#### Load Network of Amsterdam and built calculate the mean distance to the center 

In [ ]:
city_list = []
dist_citys = []
distance_progress = []
for file in os.listdir(path):
    #print(file.replace('.pkl',''))
    data = pd.read_pickle(path + '/' + file)
    city = file.replace('.pkl', '')
    city_list.append(city)

    Gu = pickle.load( open( "SN/" + 'network_graph_POI_' + city + ".p", "rb" ) )
#     for u, v, key, data in Gu.edges(keys=True, data=True):
#             if data['oneway']:
#                 data['oneway'] = False
    c_node = data['center'][0]

    #every shortest distance to the cluster-center
    distance = []
    cum_distance = 0

    dist_list = []
    dist = []

    i = 0
    for node in Gu.nodes():
        try:
            di = nx.shortest_path_length(Gu, source=node, target=c_node, weight='length')
            cum_distance += di
            dist.append(di)
        except:
            i += 1
            dist.append(np.inf)
    dist_list.append(dist)
    mean_distance = cum_distance / (len(Gu.nodes())-i)
    distance.append(mean_distance)
    print('Mean distance:', mean_distance) #3626.2361939018565

    nodes = ox.graph_to_gdfs(Gu, edges=False)

    ## Cluster
    #Amount of clusters: c = 2,7 
    for c in range(2,7):
        X = pd.concat([nodes['x'], nodes['y']], axis=1)
        kmeans = KMeans(n_clusters=c, random_state=0).fit(X)
        groupings = kmeans.labels_
        centers = kmeans.cluster_centers_
        clust_dist = []
        a = pd.DataFrame(columns=range(c), index=Gu.nodes())
        a['category'] = np.nan
        b = a
        l = list(range(c))
        tn = []
        nns = []

        #Create shortest path to each center
        for i in range(c):  
            destination = (centers[i][1], centers[i][0])
            target_node = ox.get_nearest_node(Gu, destination)
            tn.append(target_node)            
            for node in Gu.nodes():
                try:
                    length = nx.shortest_path_length(Gu, source=node, target=target_node, weight='length')
                    a.loc[node,i] = length
                except:
                    a.loc[node,i] = np.inf
        #Select Cluster
        for i in range(len(a)):
            a['category'].iloc[i] = min(range(len(a.iloc[i])-1), key=a.iloc[i].__getitem__)          

        #for each cluster...
        for i in range(c):
            cl = []
            cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
            #Coordinates
            y1 = [Gu.nodes()[n]['y'] for n in cl]
            x1 = [Gu.nodes()[n]['x'] for n in cl]
            #Z-Values
            z = np.abs(zscore(list(zip(y1,x1))))
            #remove the outlier from clusters
            nodes_remove = [cl[ele] for ele in np.where(z > 2.5)[0]]
            #nodes_remove[:5] #werden nicht für die Berechnung berücksichtigt
            c_new = [j for j in cl if j not in nodes_remove]
            for node in nodes_remove:
                a['category'][node] = np.nan
            #a['category'].value_counts(dropna=False)

            #Coordinates new
            y1 = [Gu.nodes()[n]['y'] for n in cl]
            x1 = [Gu.nodes()[n]['x'] for n in cl]
            #New Center
            center_y1 = sum(y1)/len(y1)
            center_x1 = sum(x1)/len(x1)
            #Nearest Node to Center
            nn = ox.get_nearest_node(Gu, (center_y1, center_x1))
            nns.append(nn)
            #calculate mean distance of this cluster
            for node in c_new:
                try:
                    length = nx.shortest_path_length(Gu, source=node, target=nn, weight='length')
                    clust_dist.append(length)
                    b.loc[node,i] = length
                except:
                    b.loc[node,i] = np.inf 

        #calculate distance 
        j = 0
        k = 0
        for i in range(len(a)):
            if min(a[l].iloc[i]) != np.inf:
                j += min(a[l].iloc[i])
                k += 1
                dist.append(min(a[l].iloc[i]))
            else:
                dist.append(np.inf)
        dist_list.append(dist)
        mean = j/k
        distance.append(mean)
        print('Mean distance:', mean, '; ', len(a)-k, '-Values with infinite distance' )
        a['category'].value_counts(dropna=False)
        colors = ['firebrick', 'steelblue', 'olivedrab', 'goldenrod', 'mediumpurple', 'lightseagreen']
        ec = []
        for i in Gu.nodes():
            try:
                ec.append(colors[int(a.loc[i]['category'])])
            except:
                ec.append('gray')
        #ec = ['r' if a.loc[i]['category'] == 0 else 'g' for i in Gu.nodes()]
        ns = [100 if node in nns else 5 for node in Gu.nodes()]
        fig, ax = ox.plot_graph(Gu, node_size=ns, bgcolor='w', node_color=ec, edge_linewidth=0.4, edge_alpha=1, show=False, close=False)
        ax.set_title(file.replace('.pkl', '') + ' in ' + str(c) + ' Cluster', size=14)
        plt.tight_layout()
        plt.savefig('Plots/cluster' + str(c) + '_' + city + '.pdf')
        plt.show()
    dist_citys.append(dist_list)
    distance_progress.append(distance)
open_file = open('distance_progress_cities.pkl', "wb")
pickle.dump(distance_progress, open_file)
open_file.close()

open_file = open('distance_distribution_cities.pkl', "wb")
pickle.dump(dist_citys, open_file)
open_file.close()

In [ ]:
with open('distance_distribution_cities.pkl', 'rb') as f:
    dist = pickle.load(f)
with open('distance_progress_cities.pkl', "rb") as f:
    progr = pickle.load(f)

In [ ]:
progr